In [1]:
import os

from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext

os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.1.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0 pyspark-shell'


In [2]:
kafka_broker_hostname = 'kafka-49b7861-rafaelathaydemello-3b01.aivencloud.com'
kafka_broker_port = '25697'
kafka_broker = kafka_broker_hostname + ':' +kafka_broker_port
kafka_topic= 'picture-metadata'

spark = SparkSession.builder.appName('PicturesAnalysis').getOrCreate()


In [3]:
# Batch Query
df = spark.read.format('kafka') \
    .option('kafka.bootstrap.servers', kafka_broker)\
    .option("kafka.security.protocol","SSL")\
    .option("kafka.ssl.endpoint.identification.algorithm", "")\
    .option("kafka.ssl.truststore.location", "/home/jovyan/local/client.truststore.jks")\
    .option("kafka.ssl.truststore.password", "password")\
    .option("kafka.ssl.keystore.type", "PKCS12")\
    .option('kafka.ssl.keystore.location', "/home/jovyan/local/client.keystore.p12")\
    .option('kafka.ssl.keystore.password', "password")\
    .option('subscribe', kafka_topic)\
    .option('startingOffsets', 'earliest')\
    .option("includeHeaders", "true")\
    .load()


In [4]:
spark.catalog.listTables()

[]

In [5]:
a = df.createOrReplaceTempView('test')

In [6]:
df2 = spark.sql("SELECT CAST(value as string) FROM test")

In [7]:
df2.printSchema()


root
 |-- value: string (nullable = true)



In [8]:
df2.show()

+--------------------+
|               value|
+--------------------+
|{"success": true,...|
|{"success": true,...|
|{"success": true,...|
|{"success": true,...|
|{"success": true,...|
|{"success": true,...|
|{"success": true,...|
|{"success": true,...|
|{"success": true,...|
|{"success": true,...|
|{"success": true,...|
|{"success": true,...|
|{"success": true,...|
|{"success": true,...|
|{"success": true,...|
|{"success": true,...|
|{"success": true,...|
|{"success": true,...|
|{"success": true,...|
|{"success": true,...|
+--------------------+
only showing top 20 rows

